# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs vroom iinput output formats:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

http://localhost/notebooks/tests/test5.ipynb 2 veicoli 1 solo mattina 2 corse

http://localhost/notebooks/tests/test6.ipynb 2 veicoli tutto il giorno 4 corse con priorita


In [40]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time

vroom_url = 'http://vroom:3000/optimization/'
vehicleProfile = "driving-car"

# DATI posizioni
geoA = json.load(open('../data/mezzolombardo.geojson'))['features']   #4 points
geoB = json.load(open('../data/mezzocorona.geojson'))['features']     #6 points
geoC = json.load(open('../data/piana_rotaliana.geojson'))['features'] #6 points distanti

pointsA = [ p['geometry']['coordinates'] for p in geoA ]
pointsB = [ p['geometry']['coordinates'] for p in geoB ]
pointsC = [ p['geometry']['coordinates'] for p in geoC ]

#drt_example = json.load(open('../data/drt_2_veicoli_4_corse.json'))
drt_example = json.load(open('../data/drt_2_veicoli-fulltime_4_corse-priorita.json'))

for v in drt_example['vehicles']:
    v['profile'] = vehicleProfile

center = [
    11.1126,
    46.2083
]
zoom = 14

mapBbox = [11.077569,46.187199,11.161166,46.230027]

point_start =  [11.10249,46.21579] #posizione iniziale veicoli
point_start2 = [11.11258,46.20834]
point_end = point_start.copy()

def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]

def randomPoint(bb):
    min_x, min_y, max_x, max_y = bb
    x = random.uniform(min_x, max_x)
    y = random.uniform(min_y, max_y)
    return [x, y]

def vehicle2Map(v):
    idv = str(v['id'])    
    loc = v['start']
    cap = str(v['capacity'][0])

    try:
        times = v['time_window']
    except:
        times = []
    
    mText = "VEICOLO %s" % idv
    #mText += "<br> capacity %s" % cap
    
    if len(times) > 0:
        mText += "<br>Orario turno <br> da %s a %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = 'red'#COLORS[idv]
    
    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right',permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

def job2Map(j):
    idj = str(j['id'])    
    loc = j['location']
    if hasattr(j,'time_windows'):
        times = j['time_windows'][0]
    else:
        times = []
    
    mText = "job %s<br>" % (idj)
    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "da: %s <br> a: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )

    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right', permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

def ship2Map(s):

    color = COLORS[str(s['pickup']['id'])]
    
    # prenotazione PARTENZA
    loc1 = s['pickup']['location']
    mText = "ID %s" % s['pickup']['id']
    if s['priority']>0:
        mText +=" (CONFERMATA)"
    
    try:
        times = s['pickup']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "<br>Partenza orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    #mText += "Quantita %s <br>" % s['amount'][0]
    #mText += "Priority %s <br>" % s['priority']
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True) 
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc1), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)
    
    # prenotazione ARRIVO
    loc2 = s['delivery']['location']
    mText = "ID %s <br>" % s['delivery']['id']
    try:
        times = s['delivery']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "Arrivo orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )

    #mText += "Quantita %s <br>" % s['amount'][0]
    
    folium.PolyLine(locations = [revcc(loc1),revcc(loc2)], color = color, opacity = 0.6, weight = 3).add_to(Map)
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True)
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc2), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)

#CUSTOM ICONS https://fontawesome.com/icons?d=gallery&p=2

Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

COLORS = {
    '1':'blue','2':'darkred','3':'purple','4':'green',
    '5':'orange','6':'cadetblue','7':'beige','8':'pink',
    '9':'darkblue','10':'darkgreen'
}

HOURS1 = [
    [
        hour2sec('08:00'),
        hour2sec('08:05')
    ],
    [
        hour2sec('09:00'),
        hour2sec('09:05')
    ],
    [
        hour2sec('10:00'),
        hour2sec('10:05')
    ],
    [
        hour2sec('11:00'),
        hour2sec('11:05')
    ],
    [
        hour2sec('12:00'),
        hour2sec('12:05')
    ]
]

HOURS2 = [
    [
        hour2sec('14:00'),
        hour2sec('14:05')
    ],
    [
        hour2sec('15:00'),
        hour2sec('15:05')
    ],
    [
        hour2sec('16:00'),
        hour2sec('16:05')
    ],
    [
        hour2sec('17:00'),
        hour2sec('17:05')
    ],
    [
        hour2sec('18:00'),
        hour2sec('18:05')
    ]
]
# VEICOLI

HOURSall = HOURS1 + HOURS2

vehicles = []

### VEICOLO 1
vid = 1

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [1],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,       # posizione di partenza veicoli
    'end': loc,
    'time_window': [              #orario lavoro del veicolo 1
        HOURS1[0][0], HOURS1[-1][-1]
    ],
#     'breaks': [
#         {
#           "id": 1,
#           "service": 300,   #durata pausa 5 minuti
#           "time_windows": [ #an array of time_window objects describing valid slots for break start
#             [hour2sec('13:05'),hour2sec('13:55')],
#           ]
#         }
#     ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2

loc = drt_example['vehicles'][1]['start']

vehicle = {
    'id': vid,
    'capacity': [1],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,        # posizione di partenza veicoli
    'end': loc,
    'time_window': [                  # pomeridiano
        HOURS2[0][0], HOURS2[-1][-1]
    ]
}
vehicles.append(vehicle)

points = pointsA + pointsB


shipsProgrammed = [1, 2, 3]    # SIMULA ELENCO PRENOTAZIONI PROGRAMMATE( elenco id prenotazioni)

shipments = []

for sid in list(range(1,5)):
    
    ship = {          #struttura minima valida per prenotazione
        'delivery': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },
        'pickup': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },
        'priority': 0,
        'amount': [ 1 ]        
    }
    
    if sid in shipsProgrammed:
        ship['priority']= 100;
        
    shiptype = random.choice(['pickup','delivery']) # pickup == PARTENZA, delivery == ARRIVO
    
    ship[ shiptype ]['time_windows'] = [ random.choice(HOURSall) ]
    
    shipments.append(ship)

request = {
     'vehicles': vehicles,
     'jobs': [],
     'shipments': shipments
}

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for v in request['vehicles']:
    vehicle2Map(v)

for s in request['shipments']:
    #print(json.dumps(s, indent=4))
    ship2Map(s)

print(len(shipments))
print(shipments)

Map

4
[{'delivery': {'id': 1, 'location': [11.092131184046115, 46.221249591582584]}, 'pickup': {'id': 1, 'location': [11.125181313701608, 46.224107156263216], 'time_windows': [[1617984000, 1617984300]]}, 'priority': 100, 'amount': [1]}, {'delivery': {'id': 2, 'location': [11.12662049632712, 46.20471812180379]}, 'pickup': {'id': 2, 'location': [11.146832304991742, 46.22632610481316], 'time_windows': [[1617969600, 1617969900]]}, 'priority': 100, 'amount': [1]}, {'delivery': {'id': 3, 'location': [11.137356031075722, 46.19917822166239], 'time_windows': [[1617966000, 1617966300]]}, 'pickup': {'id': 3, 'location': [11.099248992944752, 46.19191965370271]}, 'priority': 100, 'amount': [1]}, {'delivery': {'id': 4, 'location': [11.150102718072638, 46.208254657712715]}, 'pickup': {'id': 4, 'location': [11.093615903530466, 46.192053247695156], 'time_windows': [[1617958800, 1617959100]]}, 'priority': 0, 'amount': [1]}]


In [27]:
request['geometry'] = True       #RITORNA GEOMETRIA DELLA SOLUZIONE

resp = requests.post(vroom_url, json = request)

solution = resp.json()

#solution['summary']
solution

{'code': 0,
 'summary': {'cost': 2222,
  'unassigned': 6,
  'delivery': [2],
  'amount': [2],
  'pickup': [2],
  'service': 0,
  'duration': 2222,
  'waiting_time': 0,
  'priority': 200,
  'distance': 21789,
  'computing_times': {'loading': 17, 'solving': 3, 'routing': 28}},
 'unassigned': [{'id': 5, 'location': [11.095649415418004, 46.19579049801972]},
  {'id': 5, 'location': [11.13201776762366, 46.216575432486216]},
  {'id': 1, 'location': [11.099391786690548, 46.22985805655739]},
  {'id': 1, 'location': [11.140157708732554, 46.200128230570634]},
  {'id': 2, 'location': [11.149228628314724, 46.20345920680555]},
  {'id': 2, 'location': [11.10481310660932, 46.21960911251475]}],
 'routes': [{'vehicle': 1,
   'cost': 1556,
   'delivery': [1],
   'amount': [1],
   'pickup': [1],
   'service': 0,
   'duration': 1556,
   'waiting_time': 0,
   'priority': 200,
   'distance': 13935,
   'steps': [{'type': 'start',
     'location': [11.10649, 46.21979],
     'load': [0],
     'arrival': 1617958

In [34]:
#MAP RESULT
#Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

for v in request['vehicles']:
    vehicle2Map(v)

if len(solution['routes']) > 0:
    for route in solution['routes']:
        
        rtext = "distance: %s delivered quantity: %s pickup quantity: %s "  % ( toKm(route['distance']), route['delivery'][0], route['pickup'][0] )
        
        rtext = rtext+", durata: %s " % toMin(route['duration']+route['service'])
        
        rcolor = COLORS[str(route['vehicle'])]
        #rcolor = random.choice( list(COLORS.values()) )
        
        print('Vehicle'+str(route['vehicle']), rcolor, rtext)
        
        folium.PolyLine(
            locations = [revcc(coords) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
            color = rcolor,
            opacity = 0.8,
            weight = 5
        ).add_to(Map)
        
        i=0
        for step in route['steps']:
            i += 1
            try:
                loc = step['location']
            except:
                loc = None
            
            sText = "CORSA %s Arrivo %s" % (step['id'], sec2hour(step['arrival']))
            
            print(sText)
            
            if loc and step['type'] != 'end':
                tit = str(i)+" "+step['type']+"<br> arrivo: %s " % sec2hour(step['arrival'])
                
                mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(tit, direction='bottom', permanent=True, style='background:yellow'), icon=folium.Icon(icon='circle', prefix='fa', color='green'))
                mark.add_to(Map)
            
    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))
    
Map

Vehicle1 blue distance: 17km delivered quantity: 2 pickup quantity: 2 , durata: 21min 


KeyError: 'id'

In [30]:
print(json.dumps(solution, indent=2))

{
  "code": 0,
  "summary": {
    "cost": 1275,
    "unassigned": 4,
    "delivery": [
      2
    ],
    "amount": [
      2
    ],
    "pickup": [
      2
    ],
    "service": 0,
    "duration": 1275,
    "waiting_time": 2747,
    "priority": 400,
    "distance": 16728,
    "computing_times": {
      "loading": 17,
      "solving": 3,
      "routing": 23
    }
  },
  "unassigned": [
    {
      "id": 3,
      "location": [
        11.08717918395996,
        46.214169599073806
      ]
    },
    {
      "id": 3,
      "location": [
        11.122541427612305,
        46.20986352455638
      ]
    },
    {
      "id": 4,
      "location": [
        11.123356819152832,
        46.21419929496722
      ]
    },
    {
      "id": 4,
      "location": [
        11.094388961791992,
        46.22753112890025
      ]
    }
  ],
  "routes": [
    {
      "vehicle": 1,
      "cost": 1275,
      "delivery": [
        2
      ],
      "amount": [
        2
      ],
      "pickup": [
        2
   